# 在岸美元兑人民币汇率和沪深300指数

In [38]:
recent_days = 3

import os
import pandas as pd

# 获取当前工作目录
cwd = os.getcwd()

# 判断是否存在figure文件夹，如果不存在就创建该文件夹
if not os.path.exists('figure'):
    os.makedirs('figure')

import pandas as pd

#USDCNY_table = pd.read_csv("USDCNY.CSV", parse_dates=True, encoding='gb2312')
USDCNY_table = pd.read_csv("data_csv/USDCNY.CSV", parse_dates=True)
USDCNY_table = USDCNY_table.loc[:,['日期', '开盘价(元)', '最高价(元)', '最低价(元)', '收盘价(元)']].dropna()
USDCNY_table.columns = ['Time','Open','High','Low','Close']
USDCNY_table['Time'] = pd.to_datetime(USDCNY_table['Time'])

SH_table = pd.read_csv("data_csv/CSI300.CSV", parse_dates=True)
SH_table = SH_table.loc[:,['日期', '开盘价(元)', '最高价(元)', '最低价(元)', '收盘价(元)']].dropna()
SH_table.columns = ['Time','Open','High','Low','Close']
SH_table['Time'] = pd.to_datetime(SH_table['Time'])

merged_table = pd.merge(SH_table, USDCNY_table, on='Time', how='inner')
merged_table.sort_values(by='Time', inplace=True)
merged_table

,Time,Open_x,High_x,Low_x,Close_x,Open_y,High_y,Low_y,Close_y
0,2022-12-20 10:15:00,3856.87,3856.87,3854.09,3854.13,6.9840,6.9840,6.9818,6.9818
1,2022-12-20 10:16:00,3853.79,3854.66,3852.33,3854.66,6.9840,6.9840,6.9840,6.9840
2,2022-12-20 10:17:00,3854.89,3854.89,3853.35,3854.35,6.9842,6.9842,6.9819,6.9819
3,2022-12-20 10:19:00,3853.15,3853.90,3852.42,3853.61,6.9846,6.9846,6.9823,6.9823
4,2022-12-20 10:20:00,3853.73,3853.73,3850.72,3852.24,6.9846,6.9846,6.9846,6.9846
...,...,...,...,...,...,...,...,...,...
17646,2023-07-19 11:28:00,3839.05,3839.23,3838.55,3838.90,7.2080,7.2080,7.2080,7.2080
17647,2023-07-19 11:29:00,3838.74,3839.28,3838.73,3838.98,7.2081,7.2081,7.2081,7.2081
17648,2023-07-19 11:30:00,3838.89,3838.89,3838.89,3838.89,7.2082,7.2082,7.2082,7.2082
17649,2023-07-19 13:00:00,3838.73,3839.65,3837.77,3838.50,7.2093,7.2093,7.2093,7.2093


In [39]:
from scipy.stats import pearsonr

#merged_table['Stock Returns'] = np.log(merged_table['Close_x'] / merged_table['Close_x'].shift(1))
#merged_table['FX Returns'] = np.log(merged_table['Close_y'] / merged_table['Close_y'].shift(1))
merged_table = merged_table.dropna()

r,pvalue = pearsonr(merged_table['Close_x'], merged_table['Close_y'])
r

-0.8071082589865606

In [40]:
#import matplotlib.pyplot as plt

# plt.rcParams['font.sans-serif'] = ['SimHei']
# plt.rcParams['axes.unicode_minus'] = False

import datetime as dt

end_date = merged_table['Time'].iloc[-1]
start_date = end_date - dt.timedelta(days=recent_days)
merged_table = merged_table[(merged_table['Time'] >= start_date) & (merged_table['Time'] <= end_date)]

from pyecharts.commons.utils import JsCode
from pyecharts.charts import Line
from pyecharts import options as opts

line = Line() #创建Line实例

# 设置折线1
line.add_xaxis(merged_table['Time'].tolist())
line.add_yaxis('在岸人民币汇率', merged_table['Close_y'].tolist(), label_opts=opts.LabelOpts(is_show=False), color='#FFB6C1')

# 设置折线2，并将yaxis_index设置为1，表示使用右侧y轴
line.add_yaxis('沪深300', merged_table['Close_x'].tolist(), yaxis_index=1, label_opts=opts.LabelOpts(is_show=False), color='#6495ED')

# 设置y轴1最大值、最小值和间隔
line.set_global_opts(
    yaxis_opts=opts.AxisOpts(
        axislabel_opts=opts.LabelOpts(
            formatter=JsCode(
                "function(value){return value.toFixed(4)}"
            )
        ),
        splitline_opts=opts.SplitLineOpts(is_show=True),
        min_=merged_table['Close_y'].mean() - 3*merged_table['Close_y'].std(),  # 左侧y轴最小值
        max_=merged_table['Close_y'].mean() + 3*merged_table['Close_y'].std(),  # 左侧y轴最大值
        interval=0.1  # 左侧y轴间隔
    ),
    datazoom_opts=[
        opts.DataZoomOpts(
            is_show=True, type_="inside", xaxis_index=[0, 0],#filter_mode = 'weakFilter',
            ),
            opts.DataZoomOpts(
                is_show=True, type_="slider",xaxis_index=[0, 0], pos_top="93%",#filter_mode = 'weakFilter',
                ),
                ],
    tooltip_opts=opts.TooltipOpts(
        trigger="axis",
        axis_pointer_type="cross",
        background_color="rgba(245, 245, 245, 0.8)",
        border_width=1,
        border_color="#ccc",
        textstyle_opts=opts.TextStyleOpts(color="#000"),
        ),
    axispointer_opts=opts.AxisPointerOpts(
        is_show=True, link=[{"xAxisIndex": [0]},{"yAxisIndex": [0]},]
        ),
)

# 设置右侧y轴，并添加is_inverse=True参数
line.extend_axis(
    yaxis=opts.AxisOpts(
        axislabel_opts=opts.LabelOpts(
            formatter=JsCode(
                "function(value){return value.toFixed(2)}"
            )
        ),
        name='沪深300',
        min_=merged_table['Close_x'].mean() - 3*merged_table['Close_x'].std(),
        max_=merged_table['Close_x'].mean() + 3*merged_table['Close_x'].std(),
        interval=100,
        position='right',
        is_inverse=True
    )
)

# 隐藏数据系列标签，使图表更加简洁
line.set_series_opts(label_opts=opts.LabelOpts(is_show=False))

line.render_notebook()

In [41]:
line.render('figure/CSI300-USDCNY.html')

'c:\\Users\\siruo\\Downloads\\每周工作\\汇率股指相关性\\figure\\CSI300-USDCNY.html'

In [42]:
r,pvalue = pearsonr(merged_table['Close_x'], merged_table['Close_y'])
r

-0.8095631001950218

# 离岸美元兑人民币汇率和美元指数

In [43]:
USDX_table = pd.read_csv("data_csv/USDX.CSV", parse_dates=True)
USDX_table = USDX_table.loc[:,['日期', '开盘价(元)', '最高价(元)', '最低价(元)', '收盘价(元)']].dropna()
USDX_table.columns = ['Time','Open','High','Low','Close']
#USDX_table['Time'] = pd.to_datetime(USDX_table['Time'])
USDX_table['Time'] = pd.to_datetime(USDX_table['Time']) + dt.timedelta(hours=12) # 夏令时

USDCNH_table = pd.read_csv("data_csv/USDCNH.CSV", parse_dates=True)
USDCNH_table = USDCNH_table.loc[:,['日期', '开盘价(元)', '最高价(元)', '最低价(元)', '收盘价(元)']].dropna()
USDCNH_table.columns = ['Time','Open','High','Low','Close']
USDCNH_table['Time'] = pd.to_datetime(USDCNH_table['Time']) + dt.timedelta(hours=12)

merged_table2 = pd.merge(USDX_table, USDCNH_table, on='Time', how='outer')
merged_table2.sort_values(by='Time', inplace=True)
merged_table2

,Time,Open_x,High_x,Low_x,Close_x,Open_y,High_y,Low_y,Close_y
29516,2023-06-19 21:12:00,NaN,NaN,NaN,NaN,7.1652,7.1654,7.1647,7.1654
29517,2023-06-19 21:13:00,NaN,NaN,NaN,NaN,7.1654,7.1659,7.1650,7.1655
29518,2023-06-19 21:14:00,NaN,NaN,NaN,NaN,7.1653,7.1661,7.1652,7.1657
29519,2023-06-19 21:15:00,NaN,NaN,NaN,NaN,7.1656,7.1657,7.1655,7.1656
29520,2023-06-19 21:16:00,NaN,NaN,NaN,NaN,7.1656,7.1657,7.1655,7.1657
...,...,...,...,...,...,...,...,...,...
29511,2023-07-19 12:57:00,100.0763,100.0791,100.0681,100.0702,7.2181,7.2181,7.2175,7.2177
29512,2023-07-19 12:58:00,100.0766,100.0800,100.0663,100.0737,7.2175,7.2184,7.2174,7.2178
29513,2023-07-19 12:59:00,100.0712,100.0752,100.0639,100.0640,7.2173,7.2176,7.2172,7.2175
29514,2023-07-19 13:00:00,100.0677,100.0709,100.0626,100.0644,NaN,NaN,NaN,NaN


In [44]:
r,pvalue = pearsonr(merged_table2.dropna()['Close_x'], merged_table2.dropna()['Close_y'])
r

0.829051633096942

In [45]:
end_date = merged_table2['Time'].iloc[-1]
start_date = end_date - dt.timedelta(days=recent_days)
merged_table2 = merged_table2[(merged_table2['Time'] >= start_date) & (merged_table2['Time'] <= end_date)]


line = Line() #创建Line实例

# 设置折线1
line.add_xaxis(merged_table2['Time'].tolist())
line.add_yaxis('离岸人民币汇率', merged_table2['Close_y'].tolist(), label_opts=opts.LabelOpts(is_show=False), color='#FFB6C1')

# 设置折线2，并将yaxis_index设置为1，表示使用右侧y轴
line.add_yaxis('美元指数', merged_table2['Close_x'].tolist(), yaxis_index=1, label_opts=opts.LabelOpts(is_show=False), color='#6495ED')

# 设置y轴1最大值、最小值和间隔
line.set_global_opts(
    yaxis_opts=opts.AxisOpts(
        axislabel_opts=opts.LabelOpts(
            formatter=JsCode(
                "function(value){return value.toFixed(4)}"
            )
        ),
        splitline_opts=opts.SplitLineOpts(is_show=True),
        min_=merged_table2.dropna()['Close_y'].mean() - 3.4*merged_table2.dropna()['Close_y'].std(),  # 左侧y轴最小值
        max_=merged_table2.dropna()['Close_y'].mean() + 3.4*merged_table2.dropna()['Close_y'].std(),  # 左侧y轴最大值
        interval=0.01  # 左侧y轴间隔
    ),
    datazoom_opts=[
        opts.DataZoomOpts(
            is_show=True, type_="inside", xaxis_index=[0, 0],#filter_mode = 'weakFilter',
            ),
            opts.DataZoomOpts(
                is_show=True, type_="slider",xaxis_index=[0, 0], pos_top="93%",#filter_mode = 'weakFilter',
                ),
                ],
    tooltip_opts=opts.TooltipOpts(
        trigger="axis",
        axis_pointer_type="cross",
        background_color="rgba(245, 245, 245, 0.8)",
        border_width=1,
        border_color="#ccc",
        textstyle_opts=opts.TextStyleOpts(color="#000"),
        ),
    axispointer_opts=opts.AxisPointerOpts(
        is_show=True, link=[{"xAxisIndex": [0]},{"yAxisIndex": [0]},]
        ),
)

# 设置右侧y轴，并添加is_inverse=True参数
line.extend_axis(
    yaxis=opts.AxisOpts(
        axislabel_opts=opts.LabelOpts(
            formatter=JsCode(
                "function(value){return value.toFixed(4)}"
            )
        ),
        name='美元指数',
        min_=merged_table2.dropna()['Close_x'].mean() - 3.4*merged_table2.dropna()['Close_x'].std(),
        max_=merged_table2.dropna()['Close_x'].mean() + 3.4*merged_table2.dropna()['Close_x'].std(),
        interval=0.1,
        position='right',
        #is_inverse=True,
    )
)

# 隐藏数据系列标签，使图表更加简洁
line.set_series_opts(label_opts=opts.LabelOpts(is_show=False))

line.render_notebook()

In [46]:
line.render('figure/USDX-USDCNH.html')

'c:\\Users\\siruo\\Downloads\\每周工作\\汇率股指相关性\\figure\\USDX-USDCNH.html'

In [47]:
r,pvalue = pearsonr(merged_table2.dropna()['Close_x'], merged_table2.dropna()['Close_y'])
r

0.4062853727753381

# 离岸美元兑人民币汇率和标准普尔500指数

In [48]:
SPX_table = pd.read_csv("data_csv/SPX.GI.CSV", parse_dates=True)
SPX_table = SPX_table.loc[:,['日期', '开盘价(元)', '最高价(元)', '最低价(元)', '收盘价(元)']].dropna()
SPX_table.columns = ['Time','Open','High','Low','Close']
SPX_table['Time'] = pd.to_datetime(SPX_table['Time']) + dt.timedelta(hours=12) # 夏令时

merged_table3 = pd.merge(SPX_table, USDCNH_table, on='Time', how='inner')
merged_table3

,Time,Open_x,High_x,Low_x,Close_x,Open_y,High_y,Low_y,Close_y
0,2023-06-20 21:30:00,4396.11,4396.11,4388.26,4390.29,7.1844,7.1844,7.1827,7.1830
1,2023-06-20 21:31:00,4390.53,4392.76,4389.63,4391.62,7.1828,7.1829,7.1822,7.1823
2,2023-06-20 21:32:00,4391.71,4393.27,4391.71,4393.27,7.1822,7.1827,7.1806,7.1807
3,2023-06-20 21:33:00,4393.41,4394.95,4393.24,4394.62,7.1807,7.1812,7.1803,7.1806
4,2023-06-20 21:34:00,4394.86,4396.91,4394.83,4396.69,7.1806,7.1806,7.1798,7.1798
...,...,...,...,...,...,...,...,...,...
7295,2023-07-19 03:57:00,4555.51,4556.91,4555.00,4556.30,7.1952,7.1952,7.1952,7.1952
7296,2023-07-19 03:58:00,4556.44,4556.44,4555.03,4555.18,7.1952,7.1952,7.1952,7.1952
7297,2023-07-19 03:59:00,4555.35,4556.44,4554.39,4555.64,7.1953,7.1954,7.1953,7.1954
7298,2023-07-19 04:00:00,4555.35,4555.35,4555.10,4555.10,7.1956,7.1956,7.1954,7.1954


In [49]:
merged_table3 = merged_table3.dropna()

r,pvalue = pearsonr(merged_table3['Close_x'], merged_table3['Close_y'])
r

-0.45111644776368987

In [50]:
end_date = merged_table3['Time'].iloc[-1]
start_date = end_date - dt.timedelta(days=recent_days)
merged_table3 = merged_table3[(merged_table3['Time'] >= start_date) & (merged_table3['Time'] <= end_date)]



line = Line() #创建Line实例

# 设置折线1
line.add_xaxis(merged_table3['Time'].tolist())
line.add_yaxis('离岸人民币汇率', merged_table3['Close_y'].tolist(), label_opts=opts.LabelOpts(is_show=False), color='#FFB6C1')

# 设置折线2，并将yaxis_index设置为1，表示使用右侧y轴
line.add_yaxis('标普500', merged_table3['Close_x'].tolist(), yaxis_index=1, label_opts=opts.LabelOpts(is_show=False), color='#6495ED')

# 设置y轴1最大值、最小值和间隔
line.set_global_opts(
    yaxis_opts=opts.AxisOpts(
        axislabel_opts=opts.LabelOpts(
            formatter=JsCode(
                "function(value){return value.toFixed(4)}"
            )
        ),
        splitline_opts=opts.SplitLineOpts(is_show=True),
        min_=merged_table3['Close_y'].mean() - 3*merged_table3['Close_y'].std(),  # 左侧y轴最小值
        max_=merged_table3['Close_y'].mean() + 3*merged_table3['Close_y'].std(),  # 左侧y轴最大值
        interval=0.005  # 左侧y轴间隔
    ),
    datazoom_opts=[
        opts.DataZoomOpts(
            is_show=True, type_="inside", xaxis_index=[0, 0],#filter_mode = 'weakFilter',
            ),
            opts.DataZoomOpts(
                is_show=True, type_="slider",xaxis_index=[0, 0], pos_top="93%",#filter_mode = 'weakFilter',
                ),
                ],
    tooltip_opts=opts.TooltipOpts(
        trigger="axis",
        axis_pointer_type="cross",
        background_color="rgba(245, 245, 245, 0.8)",
        border_width=1,
        border_color="#ccc",
        textstyle_opts=opts.TextStyleOpts(color="#000"),
        ),
    axispointer_opts=opts.AxisPointerOpts(
        is_show=True, link=[{"xAxisIndex": [0]},{"yAxisIndex": [0]},]
        ),
)

# 设置右侧y轴，并添加is_inverse=True参数
line.extend_axis(
    yaxis=opts.AxisOpts(
        axislabel_opts=opts.LabelOpts(
            formatter=JsCode(
                "function(value){return value.toFixed(2)}"
            )
        ),
        name='标普500',
        min_=merged_table3['Close_x'].mean() - 3*merged_table3['Close_x'].std(),
        max_=merged_table3['Close_x'].mean() + 3*merged_table3['Close_x'].std(),
        interval=10,
        position='right',
        # is_inverse=True,
    )
)

# 隐藏数据系列标签，使图表更加简洁
line.set_series_opts(label_opts=opts.LabelOpts(is_show=False))

line.render_notebook()

In [51]:
line.render('figure/SPX.GI-USDCNH.html')

'c:\\Users\\siruo\\Downloads\\每周工作\\汇率股指相关性\\figure\\SPX.GI-USDCNH.html'

In [52]:
r,pvalue = pearsonr(merged_table3['Close_x'], merged_table3['Close_y'])
r

0.7949213433192386